# LangChain RAG Project with Pride and Prejudice book

## Load ENV variables

In [1]:
import langchain
print(langchain.__version__)

0.3.0


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
import torch
import openai

In [4]:
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

/Users/elviramagallanes/.pyenv/versions/langchain_rag/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [5]:
openai_api_key = os.getenv('OPENAI_API_KEY')

## Documents loaders

In [6]:
loader = TextLoader('Data/pride_and_prejudice.txt', encoding='utf-8')
documents = loader.load()

### Text splitting

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)
docs = text_splitter.split_documents(documents)

### Embeddings

In [8]:
embeddings = OpenAIEmbeddings()

/var/folders/q8/3qs0qblx57384mws5h46m82m0000gn/T/ipykernel_21585/1876906078.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(
/Users/elviramagallanes/.pyenv/versions/langchain_rag/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/elviramagallanes/.pyenv/versions/langchain_rag/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set 

### Create our vector storage

In [9]:
faiss_vectorstore = FAISS.from_documents(docs, embeddings)

In [10]:
faiss_vectorstore.save_local("faiss_index")

new_vector_store = FAISS.load_local(
    "faiss_index", embeddings, allow_dangerous_deserialization=True
)

In [11]:
llm = OpenAI()
retrieval_chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever())

## LLM (CHAT BOT IMPLEMENTATION)

In [12]:
from langchain.chat_models import ChatOpenAI  
from langchain.prompts import PromptTemplate

In [13]:
chat_model = ChatOpenAI(model="gpt-3.5-turbo")

/var/folders/q8/3qs0qblx57384mws5h46m82m0000gn/T/ipykernel_21585/2627565311.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  chat_model = ChatOpenAI(model="gpt-3.5-turbo")


In [14]:
template = (
'''Below you can find pride and prejudice book. This book it's a romantic novel, that every romantic person should know.

writings:
---------------------
{context_str}
---------------------

I will describe a scenario I’m going through in my life. Your task is the following:

1. Clearly understand the scenario I’m describing, pay attention to the context.
2. Extract a quote from the book that gives guidance on how a romantic person should handle the scenario I’m describing. You should extract the quote exactly as it is, without modifying it.

scenario: {query_str}'''
)

In [15]:
prompt = PromptTemplate(input_variables=["context_str", "query_str"], template=template)

In [16]:
user_query = "I'm really in love with my boyfriend, but I'm not sure if he feels the same way. What should I do?"

In [17]:
docs = retriever.invoke(user_query)
print(docs)

[Document(metadata={'source': 'Data/pride_and_prejudice.txt'}, page_content='of being in love with you, he is very much in love with her friend.”'), Document(metadata={'source': 'Data/pride_and_prejudice.txt'}, page_content='“I think you are in very great danger of making him as much in love with\nyou as ever.”'), Document(metadata={'source': 'Data/pride_and_prejudice.txt'}, page_content='other feelings which will probably soon drive away his regard for me.'), Document(metadata={'source': 'Data/pride_and_prejudice.txt'}, page_content='his falling in love at all is not extremely probable. Here again I')]


In [18]:
context = "\n\n".join([getattr(doc, 'page_content', '') for doc in docs])
print(context)  

of being in love with you, he is very much in love with her friend.”

“I think you are in very great danger of making him as much in love with
you as ever.”

other feelings which will probably soon drive away his regard for me.

his falling in love at all is not extremely probable. Here again I


In [19]:
prompt_input = prompt.format(context_str=context, query_str=user_query)
print(prompt_input)  

Below you can find pride and prejudice book. This book it's a romantic novel, that every romantic person should know.

writings:
---------------------
of being in love with you, he is very much in love with her friend.”

“I think you are in very great danger of making him as much in love with
you as ever.”

other feelings which will probably soon drive away his regard for me.

his falling in love at all is not extremely probable. Here again I
---------------------

I will describe a scenario I’m going through in my life. Your task is the following:

1. Clearly understand the scenario I’m describing, pay attention to the context.
2. Extract a quote from the book that gives guidance on how a romantic person should handle the scenario I’m describing. You should extract the quote exactly as it is, without modifying it.

scenario: I'm really in love with my boyfriend, but I'm not sure if he feels the same way. What should I do?


# Response